In [2]:
import numpy as np
import pandas as pd 
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
test['label']='t'
# test.head()

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer

#data prep
test=test.fillna(' ')
train=train.fillna(' ')
test['total']=test['title']+' '+test['author']+test['text']
train['total']=train['title']+' '+train['author']+train['text']

#tfidf
transformer = TfidfTransformer(smooth_idf=False)
count_vectorizer = CountVectorizer(ngram_range=(1, 2))
counts = count_vectorizer.fit_transform(train['total'].values)
tfidf = transformer.fit_transform(counts)

In [4]:
targets = train['label'].values
test_counts = count_vectorizer.transform(test['total'].values)
test_tfidf = transformer.fit_transform(test_counts)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tfidf, targets, random_state=0)

/home/gray/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:1466: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(n_samples / df) + 1


In [5]:
X_train.shape , y_train.shape

((15600, 3611325), (15600,))

In [7]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e5)
logreg.fit(X_train, y_train)
print('Accuracy of Lasso classifier on training set: {:.2f}'
     .format(logreg.score(X_train, y_train)))
print('Accuracy of Lasso classifier on test set: {:.2f}'
     .format(logreg.score(X_test, y_test)))

/home/gray/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy of Lasso classifier on training set: 1.00
Accuracy of Lasso classifier on test set: 0.98


In [8]:
y_train_pred = logreg.predict_proba(X_train)
print(y_train_pred[:,1], y_train)

[1.00000000e+00 9.99989048e-01 2.46202164e-05 ... 2.53000335e-07
 1.02082068e-08 2.45538298e-05] [1 1 0 ... 0 0 0]


In [9]:
y_test_pred =logreg.predict_proba(X_test)
print(y_test_pred[:,1], y_test)

[0.00108589 0.99996355 0.99999597 ... 0.22990449 0.9999998  0.55225337] [0 1 1 ... 0 1 0]


In [10]:
targets = train['label'].values
logreg = LogisticRegression()
logreg.fit(counts, targets)

example_counts = count_vectorizer.transform(test['total'].values)
predictions = logreg.predict(example_counts)
pred=pd.DataFrame(predictions,columns=['label'])
pred['id']=test['id']
pred.groupby('label').count()

/home/gray/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


,id
label,
0,2603
1,2597


In [11]:
pred.to_csv('countvect5.csv', index=False)

In [12]:
prediction_data = pd.read_csv('/home/gray/Desktop/NeuralNews/dataset-for-model/first.csv')

In [13]:
prediction_data.head()

,Title,Story Url,Brief Intro,Paragraph,Image Url
0,Cuba welcomes tourists after months of being c...,https://www.indiatoday.in/lifestyle/travel/sto...,Cuba finally welcomed tourists after months of...,['Cuba rolled out the red carpet on Friday for...,https://akm-img-a-in.tosshub.com/indiatoday/im...
1,Jalaj Saxena deserves a place at Chennai Super...,https://www.indiatoday.in/sports/cricket/story...,Jalaj Saxena is a 33-year-old right-hand batsm...,['Former India wicketkeeper Deep Dasgupta feel...,https://akm-img-a-in.tosshub.com/indiatoday/im...
2,History-sheeter arrested for robbing pilot at ...,https://www.indiatoday.in/crime/story/history-...,"A history-sheeter, who was arrested for robbin...","['A history-sheeter, who was arrested for robb...",https://akm-img-a-in.tosshub.com/indiatoday/im...
3,Bundelkhand folksinger Deshraj Pateriya dies a...,https://www.indiatoday.in/lifestyle/celebrity/...,Eminent Bundelkhand folksinger Deshraj Pateriy...,['Eminent folksinger Deshraj Pateriya passed a...,https://akm-img-a-in.tosshub.com/indiatoday/im...
4,Nations League: Czech Republic-Scotland game c...,https://www.indiatoday.in/sports/football/stor...,With a staff member testing positive and seven...,"[""The Nations League game between the Czech Re...",https://akm-img-a-in.tosshub.com/indiatoday/im...


In [14]:
prediction_data['label'] = 't'
prediction_data = prediction_data.fillna(' ')
prediction_data['total'] = prediction_data['Title'] + prediction_data['Brief Intro'] + prediction_data['Paragraph']

In [15]:
pred_counts = count_vectorizer.transform(prediction_data['total'].values)
pred_tfidf = transformer.fit_transform(pred_counts)

/home/gray/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:1466: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(n_samples / df) + 1


In [16]:
predictions = logreg.predict_proba(pred_counts)
predictions

array([[0.51873276, 0.48126724],
       [0.00180459, 0.99819541],
       [0.04786925, 0.95213075],
       [0.13193   , 0.86807   ],
       [0.30259299, 0.69740701],
       [0.20809786, 0.79190214],
       [0.7166745 , 0.2833255 ],
       [0.99407351, 0.00592649],
       [0.76370723, 0.23629277],
       [0.1824132 , 0.8175868 ],
       [0.38333772, 0.61666228],
       [0.1319462 , 0.8680538 ]])

In [17]:
prediction_data['label'] = predictions[:, [0]]*100

In [18]:
prediction_data

,Title,Story Url,Brief Intro,Paragraph,Image Url,label,total
0,Cuba welcomes tourists after months of being c...,https://www.indiatoday.in/lifestyle/travel/sto...,Cuba finally welcomed tourists after months of...,['Cuba rolled out the red carpet on Friday for...,https://akm-img-a-in.tosshub.com/indiatoday/im...,51.873276,Cuba welcomes tourists after months of being c...
1,Jalaj Saxena deserves a place at Chennai Super...,https://www.indiatoday.in/sports/cricket/story...,Jalaj Saxena is a 33-year-old right-hand batsm...,['Former India wicketkeeper Deep Dasgupta feel...,https://akm-img-a-in.tosshub.com/indiatoday/im...,0.180459,Jalaj Saxena deserves a place at Chennai Super...
2,History-sheeter arrested for robbing pilot at ...,https://www.indiatoday.in/crime/story/history-...,"A history-sheeter, who was arrested for robbin...","['A history-sheeter, who was arrested for robb...",https://akm-img-a-in.tosshub.com/indiatoday/im...,4.786925,History-sheeter arrested for robbing pilot at ...
3,Bundelkhand folksinger Deshraj Pateriya dies a...,https://www.indiatoday.in/lifestyle/celebrity/...,Eminent Bundelkhand folksinger Deshraj Pateriy...,['Eminent folksinger Deshraj Pateriya passed a...,https://akm-img-a-in.tosshub.com/indiatoday/im...,13.193000,Bundelkhand folksinger Deshraj Pateriya dies a...
4,Nations League: Czech Republic-Scotland game c...,https://www.indiatoday.in/sports/football/stor...,With a staff member testing positive and seven...,"[""The Nations League game between the Czech Re...",https://akm-img-a-in.tosshub.com/indiatoday/im...,30.259299,Nations League: Czech Republic-Scotland game c...
5,Italy's winning streak ends at 11 after draw w...,https://www.indiatoday.in/sports/football/stor...,Netherlands began life after Ronald Koeman wit...,"[""Bosnia ended Italy's run of 11 successive wi...",https://akm-img-a-in.tosshub.com/indiatoday/im...,20.809786,Italy's winning streak ends at 11 after draw w...
6,China's agressive behaviour violates bilateral...,https://www.indiatoday.in/india/story/china-s-...,Defence Minister Rajnath Singh told his Chines...,"[""Defence Minister Rajnath Singh told Chinese ...",https://akm-img-a-in.tosshub.com/indiatoday/im...,71.667450,China's agressive behaviour violates bilateral...
7,Coronavirus cases in Delhi increasing due to m...,https://www.indiatoday.in/india/story/coronavi...,Chief Minister Arvind Kejriwal said on Saturda...,['Chief Minister Arvind Kejriwal said on Satur...,https://akm-img-a-in.tosshub.com/indiatoday/im...,99.407351,Coronavirus cases in Delhi increasing due to m...
8,Veteran actor Y Gee Mahendra completes 50 year...,https://www.indiatoday.in/movies/regional-cine...,"Veteran actor Y Gee Mahendra, in a recent post...","[""Veteran actor-playwriter Y Gee Mahendra has ...",https://akm-img-a-in.tosshub.com/indiatoday/im...,76.370723,Veteran actor Y Gee Mahendra completes 50 year...
9,Sushant death case: Rhea bought and sold drugs...,https://www.indiatoday.in/movies/celebrities/s...,The NCB crime report says that Rhea Chakrabort...,"[""The crime report of Narcotics Control Bureau...",https://akm-img-a-in.tosshub.com/indiatoday/im...,18.241320,Sushant death case: Rhea bought and sold drugs...


In [19]:
prediction_data.to_csv("first_database.csv", index=False)

In [20]:
prediction_data["Paragraph"][0].rstrip("[]")

'[\'Cuba rolled out the red carpet on Friday for the first planeload of tourists to arrive on the Communist-run island in months as it struggles toward a post-pandemic new normalcy.\', \'The import-dependent country has been plunged into crisis and scarcity by tough US sanctions and the Covid-19 outbreak.\', \'The arrival was seen as a hopeful sign for tens of thousands of laid-off leisure industry employees, shuttered small businesses and hard-pressed residents in general. An Air Canada plane arrived at midday at the Cayo-Coco airport on the northcentral coast.\', "Air Canada Vacations, the airline\'s tour business, said it would now fly weekly to Cuba and biweekly beginning next month.", \'Cuba closed its airports in March due to the pandemic. While some hotels are open under international sanitary regulations at resorts in isolated areas such as Cayo-Coco, there is no indication when Havana and other cities might allow foreign visitors to return.\', \'Cuba has managed to control the

In [21]:
prediction_data["Paragraph"][0]

'[\'Cuba rolled out the red carpet on Friday for the first planeload of tourists to arrive on the Communist-run island in months as it struggles toward a post-pandemic new normalcy.\', \'The import-dependent country has been plunged into crisis and scarcity by tough US sanctions and the Covid-19 outbreak.\', \'The arrival was seen as a hopeful sign for tens of thousands of laid-off leisure industry employees, shuttered small businesses and hard-pressed residents in general. An Air Canada plane arrived at midday at the Cayo-Coco airport on the northcentral coast.\', "Air Canada Vacations, the airline\'s tour business, said it would now fly weekly to Cuba and biweekly beginning next month.", \'Cuba closed its airports in March due to the pandemic. While some hotels are open under international sanitary regulations at resorts in isolated areas such as Cayo-Coco, there is no indication when Havana and other cities might allow foreign visitors to return.\', \'Cuba has managed to control the

In [44]:
li = ['[', ']', '\'', '\"', '\\']

prediction_data["Paragraph"][0].strip("[]\\\','")

'Cuba rolled out the red carpet on Friday for the first planeload of tourists to arrive on the Communist-run island in months as it struggles toward a post-pandemic new normalcy.\', \'The import-dependent country has been plunged into crisis and scarcity by tough US sanctions and the Covid-19 outbreak.\', \'The arrival was seen as a hopeful sign for tens of thousands of laid-off leisure industry employees, shuttered small businesses and hard-pressed residents in general. An Air Canada plane arrived at midday at the Cayo-Coco airport on the northcentral coast.\', "Air Canada Vacations, the airline\'s tour business, said it would now fly weekly to Cuba and biweekly beginning next month.", \'Cuba closed its airports in March due to the pandemic. While some hotels are open under international sanitary regulations at resorts in isolated areas such as Cayo-Coco, there is no indication when Havana and other cities might allow foreign visitors to return.\', \'Cuba has managed to control the pa